In [ ]:
import pandas as pd
from glob import glob

# --------------------------------------------------------------
# Read single CSV file
# --------------------------------------------------------------
single_file_acc = pd.read_csv("/Users/rishanthreddy/Desktop/Tracking barbell Exercises/data/raw/MetaMotion/A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
single_file_gyr = pd.read_csv("/Users/rishanthreddy/Desktop/Tracking barbell Exercises/data/raw/MetaMotion/A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")
# --------------------------------------------------------------
# List all data in data/raw/MetaMotion
# --------------------------------------------------------------
files = glob("/Users/rishanthreddy/Desktop/Tracking barbell Exercises/data/raw/MetaMotion/*.csv")
len(files)

# --------------------------------------------------------------
# Extract features from filename
# --------------------------------------------------------------
data_path = "/Users/rishanthreddy/Desktop/Tracking barbell Exercises/data/raw/MetaMotion/"
f = files[1]

participant = f.split("-")[0].replace(data_path,"")
label = f.split("-")[1]
category = f.split("-")[2].rstrip("123").rstrip('_MetaWear_2019')
df = pd.read_csv(f)

df["participant"] = participant
df["label"] = label
df["category"] = category
  

 

# --------------------------------------------------------------
# Read all files
# --------------------------------------------------------------
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

acc_df.head()
gyr_df.head()

acc_set = 1
gyr_set = 1

for f in files:
    participant = f.split("-")[0].replace(data_path,"")
    label = f.split("-")[1]
    category = f.split("-")[2].rstrip("123").rstrip('_MetaWear_2019')
    
    df = pd.read_csv(f)
    
    df["participant"] = participant
    df["label"] = label
    df["category"] = category
    
    
    if "Accelerometer" in f:
        df["set"] = acc_set
        acc_set += 1
        acc_df = pd.concat([acc_df,df])
        
    
    if "Gyroscope" in f:
        df["set"] = gyr_set
        gyr_set += 1
        gyr_df = pd.concat([gyr_df,df])
        


acc_df[acc_df["set"] == 1]







    
    
  
    
    
# --------------------------------------------------------------
# Working with datetimes
# --------------------------------------------------------------

acc_df.info()
pd.to_datetime(df["epoch (ms)"],unit = "ms")

pd.to_datetime(df["time (01:00)"])

acc_df.index = pd.to_datetime(acc_df["epoch (ms)"],unit = "ms")
gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"],unit = "ms")

del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]

acc_df
gyr_df
# --------------------------------------------------------------
# Turn into function
# --------------------------------------------------------------


# --------------------------------------------------------------
# Merging datasets
# --------------------------------------------------------------
data_merged = pd.concat([acc_df.iloc[:,:3],gyr_df],axis =1)
data_merged.head(50)

data_merged.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
]

# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz

sampling = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last",
    "label": "last",
    "category": "last",
   
    "set": "last",
    
}
data_merged[:1000].resample(rule = "200ms").apply(sampling)

days = [g for n,g in data_merged.groupby(pd.Grouper(freq="D"))]

data_resampled = pd.concat([df.resample(rule = "200ms").apply(sampling).dropna() for df in days])
data_resampled.info()


data_resampled["set"] = data_resampled["set"].astype("int")

# --------------------------------------------------------------
# Export dataset
# --------------------------------------------------------------

data_resampled.to_pickle("/Users/rishanthreddy/Desktop/Tracking barbell Exercises/data/interim/01_data_processed.pkl")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23578 entries, 0 to 169
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   epoch (ms)    23578 non-null  int64  
 1   time (01:00)  23578 non-null  object 
 2   elapsed (s)   23578 non-null  float64
 3   x-axis (g)    23578 non-null  float64
 4   y-axis (g)    23578 non-null  float64
 5   z-axis (g)    23578 non-null  float64
 6   participant   23578 non-null  object 
 7   label         23578 non-null  object 
 8   category      23578 non-null  object 
 9   set           23578 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null 

In [ ]:
data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 774.2+ KB


In [ ]:
data_resampled.head()

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,64
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,64
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,64
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,64
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,64


In [ ]:
data_resampled(100)

TypeError: 'DataFrame' object is not callable

In [ ]:
data_resampled.head(100)

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,64
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,64
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,64
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,64
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,64
...,...,...,...,...,...,...,...,...,...,...
2019-01-11 15:10:10.400,-0.131000,0.950667,-0.149667,-2.4024,-0.7804,10.5000,A,bench,heavy,72
2019-01-11 15:10:10.600,-0.099500,0.952500,-0.119500,-8.9388,-0.2806,22.1584,A,bench,heavy,72
2019-01-11 15:10:10.800,-0.027333,0.848000,-0.114333,-9.1950,7.0854,22.0854,A,bench,heavy,72


In [7]:
data_resampled

NameError: name 'data_resampled' is not defined